# Lab | Inferential statistics - T-test & P-value

Importing libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import t
from scipy import stats

## 1. One tailed t-test

In a packing plant, a machine packs cartons with jars. It is supposed that a new machine will pack faster on the average than the machine currently used. To test that hypothesis, the times it takes each machine to pack ten cartons are recorded. The results, in seconds, are shown in the tables in the file files_for_lab/machine.txt. Assume that there is sufficient evidence to conduct the t test, does the data provide sufficient evidence to show if one machine is better than the other?

In [2]:
machine = pd.read_csv('files_for_lab/machine.txt', sep="\t", encoding="utf-16")
machine

,New machine,Old machine
0,42.1,42.7
1,41.0,43.6
2,41.3,43.8
3,41.8,43.3
4,42.4,42.5
5,42.8,43.5
6,43.2,43.1
7,42.3,41.7
8,41.8,44.0
9,42.7,44.1


In [3]:
machine.columns = ['new','old']

In [4]:
stats.ttest_ind(machine['new'], machine['old'])

Ttest_indResult(statistic=-3.3972307061176026, pvalue=0.0032111425007745158)

p-value is lower than alpha (0.05), so null hypothesis is rejected. One machine is better than the other.\
\
\
Is the new machine better than the old one?\
&emsp; H0: new >= old\
&emsp; new < old

In [5]:
stats.ttest_ind(machine['new'], machine['old'], alternative='less')

Ttest_indResult(statistic=-3.3972307061176026, pvalue=0.0016055712503872579)

p-value is lower than alpha (0.05), so null hypothesis is rejected, new machine takes significantly less time to pack than old machine.

# 2. Matched Pairs Test 

In this challenge we will compare dependent samples of data describing our Pokemon (file files_for_lab/pokemon.csv). Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. Compare the two columns to see if there is a statistically significant difference between them and comment your result.

In [6]:
pokemon = pd.read_csv('files_for_lab/pokemon.csv')
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


Is defense and attack significant similar or different for each pokemon?\
&emsp; H0: attack = defense\
&emsp; H1: attack != defense

In [7]:
stats.ttest_rel(pokemon['Attack'], pokemon['Defense'])

Ttest_relResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05)

p-value is lower than alpha (0.05), so null hypotesis is rejected.\
&emsp; H0: attack <= defense\
&emsp; H1: attack > defense

In [8]:
stats.ttest_rel(pokemon['Attack'], pokemon['Defense'], alternative='greater')

Ttest_relResult(statistic=4.325566393330478, pvalue=8.570151739679279e-06)

p-value is lower than alpha (0.05), so null hypothesis is rejected, attack is greater than defense.

# 3. 

Here are the steps that you would need to work on: - Null hypothesis - Alternate hypothesis - Level of significance - Test statistic - P-value - F table\
\
Context:\
Suppose you are working as an analyst in a microprocessor chip manufacturing plant. You have been given the task of analyzing a plasma etching process with respect to changing Power (in Watts) of the plasma beam. Data was collected and provided to you to conduct statistical analysis and check if changing the power of the plasma beam has any effect on the etching rate by the machine. You will conduct ANOVA and check if there is any difference in the mean etching rate for different levels of power. You can find the data anova_lab_data.xlsx file in the files_for_lab folder\
 \
State the null hypothesis\
State the alternate hypothesis\
What is the significance level\
What are the degrees of freedom of model, error terms, and total DoF

In [9]:
anova = pd.read_excel('files_for_lab/anova_lab_data.xlsx')
anova.head()

,Power,Etching Rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71


In [10]:
anova.columns

Index(['Power ', 'Etching Rate'], dtype='object')

In [11]:
anova.groupby('Power ').agg({'Etching Rate':np.mean})

,Etching Rate
Power,
160 W,5.792
180 W,6.238
200 W,8.318


&emsp; H0: 160 = 180 = 200\
&emsp; H1: 160 != 180 | 160 != 200 | 180 != 200

Significance level is 0.05

In [12]:
len(anova)

15

Degrees of freedom = 14

In [13]:
S2t = 0
for power in anova['Power '].unique():
    ng = len(anova[anova['Power '] == power])  
    S2t  += ( ( anova[anova['Power '] == power]['Etching Rate'].mean() - anova['Etching Rate'].mean() ) ** 2) * ng
S2t /= ( anova['Power '].nunique() - 1 )
print("The value of S2t is {:.2f}".format(S2t)) 

The value of S2t is 9.09


In [14]:
S2E = 0
for power in anova['Power '].unique():
    for rate in anova[anova['Power '] == power]['Etching Rate']:
        S2E += ( rate - anova[anova['Power '] == power]['Etching Rate'].mean() ) ** 2
S2E /= ( len(anova) - anova['Power '].nunique() ) 

print()
print("The value of S2E is {:.2f}".format(S2E))


The value of S2E is 0.25


In [15]:
F = S2t / S2E
print("The value of F is {:.2f}".format(F))

The value of F is 36.88


# 4.

In this section, use the Python to conduct ANOVA.
What conclusions can you draw from the experiment and why?

In [16]:
stats.f_oneway(anova[anova['Power '] == '160 W']['Etching Rate'],anova[anova['Power '] == '180 W']['Etching Rate'],anova[anova['Power '] == '200 W']['Etching Rate'])[1]

7.506584272358903e-06

p-value is lower than alpha (0.05), so null hypothesis is rejected, there is a difference on the means.